In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import os

In [2]:
df = pd.read_csv('normalized_score_new.csv',index_col='Unnamed: 0')

In [3]:
df2 = pd.read_csv('regression_input.csv',index_col='Unnamed: 0')

In [4]:
df = df.join(df2)
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')

In [5]:
df.columns

Index(['anger_score', 'confusion_score', 'depression_score', 'fatigue_score',
       'tension_score', 'vigour_score', 'esteem_score', 'TMD_score', 'return'],
      dtype='object')

In [6]:
df[['anger_score', 'confusion_score', 'depression_score', 'fatigue_score',
       'tension_score', 'vigour_score', 'esteem_score']].rolling(30).mean().plot(figsize=(16,9),grid=True)

In [7]:
df = df[['TMD_score', 'vigour_score', 'fatigue_score', 'return','tension_score']]

In [8]:
df.columns

Index(['TMD_score', 'vigour_score', 'fatigue_score', 'return',
       'tension_score'],
      dtype='object')

In [9]:
df['return_lag_3'] = df['return'].shift(-3)

In [10]:
def rolling_regression(a,temp):
    X = sm.add_constant(a.iloc[:60][temp])
    y = a.iloc[:60]['return_lag_3']
    model = sm.OLS(y, X).fit()
    beta = model.params[1]
    std = model.bse[1]
    pvalues = model.pvalues[1]
    r2 = 1-np.sum((a.iloc[-4:]['return_lag_3']-
                   model.predict(sm.add_constant(a.iloc[-4:][temp])))**2)/np.sum((a.iloc[-4:]['return_lag_3']-
                                                                                        np.mean(a.iloc[-4:]['return_lag_3']))**2)
    
    MAPE = np.mean(np.abs((a.iloc[-4:]['return_lag_3']-model.predict(sm.add_constant(a.iloc[-4:][temp])))/a.iloc[-4:]['return_lag_3'].values))
    
    
    
    return beta,std,r2,MAPE,pvalues

In [11]:
df.columns

Index(['TMD_score', 'vigour_score', 'fatigue_score', 'return', 'tension_score',
       'return_lag_3'],
      dtype='object')

In [12]:
df = df.dropna()
extra = []
for i in range(len(df)-64):
    [beta,std,r2,MAPE,pvalues]= rolling_regression(df.iloc[i:64+i],'TMD_score')
    extra.append([beta,std,r2,MAPE,pvalues])
extra = pd.DataFrame(extra, columns=['beta','std','r2','MAPE','pvalues'], index=df.index[60:-4])
extra['MAPE'].mean()

/Users/cactus/anaconda3/envs/stats/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2223: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


1.668919820880337

In [13]:
df = df.dropna()
extra = []
for i in range(len(df)-64):
    [beta,std,r2,MAPE,pvalues]= rolling_regression(df.iloc[i:64+i],['TMD_score','vigour_score'])
    extra.append([beta,std,r2,MAPE,pvalues])
extra = pd.DataFrame(extra, columns=['beta','std','r2','MAPE','pvalues'], index=df.index[60:-4])
extra['MAPE'].mean()

1.7646604481769315

In [14]:
df = df.dropna()
extra = []
for i in range(len(df)-64):
    [beta,std,r2,MAPE,pvalues]= rolling_regression(df.iloc[i:64+i],['TMD_score','vigour_score','fatigue_score'])
    extra.append([beta,std,r2,MAPE,pvalues])
extra = pd.DataFrame(extra, columns=['beta','std','r2','MAPE','pvalues'], index=df.index[60:-4])
extra['MAPE'].mean()

1.9238966907014936

In [15]:
df = df.dropna()
extra = []
for i in range(len(df)-64):
    [beta,std,r2,MAPE,pvalues]= rolling_regression(df.iloc[i:64+i],['return'])
    extra.append([beta,std,r2,MAPE,pvalues])
extra = pd.DataFrame(extra, columns=['beta','std','r2','MAPE','pvalues'], index=df.index[60:-4])
extra['MAPE'].mean()

1.6596196482090442

In [16]:
df = df.dropna()
extra = []
for i in range(len(df)-64):
    [beta,std,r2,MAPE,pvalues]= rolling_regression(df.iloc[i:64+i],['TMD_score','vigour_score','tension_score'])
    extra.append([beta,std,r2,MAPE,pvalues])
extra = pd.DataFrame(extra, columns=['beta','std','r2','MAPE','pvalues'], index=df.index[60:-4])
extra['MAPE'].mean()

1.7009653413942132

In [17]:
extra['band_up'] = extra['beta']+extra['std']
extra['band_down'] = extra['beta']-extra['std']

In [18]:
extra[['beta','band_up','band_down']].plot(figsize=(16,9),grid=True)

In [19]:
len(extra[extra['r2'] <= 0])/(len(extra))

0.797979797979798

In [20]:
extra['MAPE'].mean()

1.7009653413942132